## "NOT EXISTS" AND "EXISTS" equivalent operations on dataframes


In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('test_dataframes').enableHiveSupport().getOrCreate()

## Generating data

In [35]:
data1 = [(1,"Andre"),(2,"Rose")]
data2 = [(1,"Andre"),(2,"Rose"),(3,"Daniel")]
data3 = [(1,"Andre"),(2,"Rose"),(3,"Daniel"), (3,"Daniel"), (4,"Anita")]
schema = StructType([
    StructField("id",StringType(),True),
    StructField("name",StringType(),True),
])

df1 = spark.createDataFrame(data1,schema)
df2 = spark.createDataFrame(data2,schema)
df3 = spark.createDataFrame(data3,schema)
df1.show()
df2.show()
df3.show()

+---+-----+
| id| name|
+---+-----+
|  1|Andre|
|  2| Rose|
+---+-----+

+---+------+
| id|  name|
+---+------+
|  1| Andre|
|  2|  Rose|
|  3|Daniel|
+---+------+

+---+------+
| id|  name|
+---+------+
|  1| Andre|
|  2|  Rose|
|  3|Daniel|
|  3|Daniel|
|  4| Anita|
+---+------+



## NOT EXISTS EQUIVALENT

### Method 1 - subtract

In [36]:
dfr = df2.subtract(df1)
dfr.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
+---+------+



In [47]:
dfr2 = df3.subtract(df1)
dfr2.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
|  4| Anita|
+---+------+



### Method 2 - left_anti. This is the 'classical' way to have something equivalent to 'NOT EXISTS'

In [37]:
dfr = df2.join(df1,'id','left_anti')
dfr.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
+---+------+



In [54]:
dfr2 = df3.join(df1,'id','left_anti').dropDuplicates()
dfr2.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
|  4| Anita|
+---+------+



### Method 3 - exceptAll - same thing as 'subtract', apearently!

In [51]:
dfr = df2.exceptAll(df1)
dfr.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
+---+------+



In [55]:
dfr2 = df3.exceptAll(df1).dropDuplicates()
dfr2.show()

+---+------+
| id|  name|
+---+------+
|  3|Daniel|
|  4| Anita|
+---+------+



## EXISTS EQUIVALENT

### Method 1 - left_semi

In [40]:
dfr = df2.join(df1,'id','left_semi')
dfr.show()

+---+-----+
| id| name|
+---+-----+
|  1|Andre|
|  2| Rose|
+---+-----+



In [56]:
dfr = df3.join(df1,'id','left_semi')
dfr.show()

+---+-----+
| id| name|
+---+-----+
|  1|Andre|
|  2| Rose|
+---+-----+

